Pipeline 1: Thu thập Dữ liệu và Tiền xử lý ban đầu

In [55]:
import os  # Thư viện sử dụng để làm việc với hệ thống tệp và thư mục
import fitz  # Thư viện (PyMuPDF) sử dụng để đọc và xử lý các tệp PDF
import pandas as pd  # Thư viện sử dụng để quản lý và phân tích dữ liệu dưới dạng bảng (DataFrame)
from sklearn.preprocessing import LabelEncoder  # Thư viện sử dụng để mã hóa các nhãn thành giá trị số
from tensorflow.keras.preprocessing.text import Tokenizer  # Thư viện sử dụng để xử lý văn bản trong TensorFlow Keras
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Thư viện sử dụng để xử lý chuỗi trong TensorFlow Keras
from sklearn.model_selection import train_test_split  # Thư viện sử dụng để chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.utils.class_weight import compute_class_weight  # Thư viện sử dụng để tính trọng số của các lớp để xử lý vấn đề không cân bằng dữ liệu

# Hàm để trích xuất văn bản từ một file PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)  # Mở file PDF
    return " ".join([page.get_text() for page in document])  # Lấy văn bản từ mỗi trang và gộp lại thành chuỗi

In [56]:
# Trích xuất văn bản từ tất cả các file PDF trong thư mục và lưu vào CSV
pdf_folder = "C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/CV"
data = [{"filename": pdf_file, "content": extract_text_from_pdf(os.path.join(pdf_folder, pdf_file))} for pdf_file in os.listdir(pdf_folder)]
pd.DataFrame(data).to_csv("cv_texts.csv", index=False)  # Tạo DataFrame từ danh sách và lưu vào CSV

In [57]:
# Tải và chuẩn hóa dữ liệu từ các file CSV
def load_and_standardize_csv(file_path, text_col, label_col=None, default_label=0):
    df = pd.read_csv(file_path)
    df['text'] = df[text_col]
    df['label'] = df[label_col] if label_col else default_label
    return df[['text', 'label']]

category_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_category_data.csv", 'category')
encoded_category_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_encoded_category_data.csv", 'category', 'encoded_category')
jobs_classify_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_jobs_classify_data.csv", 'text', 'category')
jobs_skills_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_jobs_skills_data.csv", 'skills', 'category')
resume_skills_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_resume_skills_data.csv", 'Resume', 'Category')
new_skills_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_new_skills_data.csv", 'skills')
skill_set_df = load_and_standardize_csv("C:/Users/pngoc/Downloads/Smart-Talent-Resume-Ranker-kapil-development/Smart-Talent-Resume-Ranker-kapil-development/Datasets/cleaned_skill_set_data.csv", 'skills')

combined_df = pd.concat([category_df, encoded_category_df, jobs_classify_df, jobs_skills_df, resume_skills_df, new_skills_df, skill_set_df], ignore_index=True).dropna()

Pipeline 2: Xây dụng và huấn luyện mô hình

In [41]:
##Import các thư viện cần thiết
from tensorflow.keras.models import Sequential  # Xây dựng các mô hình tuần tự (sequential models) của Keras
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout, BatchNormalization, Bidirectional  # Các lớp của Keras để xây dựng mô hình deep learning
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping  # Cung cấp các callback để điều chỉnh tốc độ học (learning rate) và dừng sớm (early stopping) khi huấn luyện mô hình, kiểu mô hình không hiệu quả thì nó sẽ dừng

model = Sequential()  # Khởi tạo mô hình tuần tự
model.add(Embedding(input_dim=5000, output_dim=128))  # Thêm lớp Embedding để chuyển đổi các từ thành vector số
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Sử dụng Bi-LSTM (LSTM hai chiều) để mô hình học từ cả hai hướng của chuỗi
model.add(BatchNormalization())  # Thêm lớp BatchNormalization để chuẩn hóa các đầu ra của lớp trước đó
model.add(Dropout(0.5))  # Thêm lớp Dropout để ngăn chặn quá khớp (overfitting) bằng cách loại bỏ ngẫu nhiên một số đơn vị trong lớp
model.add(GRU(64))  # Thêm lớp GRU (Gated Recurrent Unit) để học các mối quan hệ trong chuỗi
model.add(BatchNormalization())  # Thêm lớp BatchNormalization để chuẩn hóa các đầu ra của lớp trước đó
model.add(Dropout(0.5))  # Thêm lớp Dropout để tránh overfitting
model.add(Dense(128, activation='relu'))  # Thêm lớp Dense (lớp dày) với hàm kích hoạt ReLU
model.add(Dropout(0.5))  # Thêm lớp Dropout để tránh overfitting
model.add(Dense(len(le.classes_), activation='softmax'))  # Thêm lớp Dense cuối cùng với hàm kích hoạt softmax để phân loại đầu ra

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)  # Callback để giảm tốc độ học khi độ lỗi không giảm
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)  # Callback để dừng sớm nếu độ lỗi không giảm sau một số lần huấn luyện

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Biên dịch mô hình với hàm mất mát (loss function) và trình tối ưu hóa (optimizer)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr, early_stopping], class_weight=class_weights_dict)  # Huấn luyện mô hình với dữ liệu huấn luyện và các callback đã thiết lập

Epoch 1/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 463s 397ms/step - accuracy: 0.0270 - loss: 5.9424 - val_accuracy: 3.3602e-04 - val_loss: 5.1911 - learning_rate: 0.0010
Epoch 2/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 332s 298ms/step - accuracy: 0.0739 - loss: 5.5406 - val_accuracy: 0.0013 - val_loss: 5.5817 - learning_rate: 0.0010
Epoch 3/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 413s 370ms/step - accuracy: 0.0617 - loss: 5.5821 - val_accuracy: 0.0062 - val_loss: 5.3401 - learning_rate: 0.0010
Epoch 4/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 478s 428ms/step - accuracy: 0.0467 - loss: 5.3975 - val_accuracy: 0.3346 - val_loss: 4.5135 - learning_rate: 2.0000e-04
Epoch 5/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 497s 445ms/step - accuracy: 0.0330 - loss: 5.1186 - val_accuracy: 0.3317 - val_loss: 4.8087 - learning_rate: 2.0000e-04
Epoch 6/10
1116/1116 ━━━━━━━━━━━━━━━━━━━━ 434s 389ms/step - accuracy: 0.0359 - loss: 5.1136 - val_accuracy: 0.3470 - val_loss: 4.5689 - learning_rate: 2.0000e-04
Epoch 7/10
1116/1116 ━━━━━━━━━━━━━━━

In [51]:
# Demo quy trình dự đoán điểm CV
def predict_matching_score(cv_text):
    cv_seq = tokenizer.texts_to_sequences([cv_text])  # Chuyển đổi văn bản CV thành chuỗi số
    cv_pad = pad_sequences(cv_seq, maxlen=100)  # Thực hiện padding (thêm) chuỗi số đến độ dài cố định
    score = model.predict(cv_pad)  # Dự đoán điểm số dựa trên mô hình đã huấn luyện
    matching_percentage = score.max() * 10000  # Chuyển đổi điểm cao nhất thành tỷ lệ phần trăm
    return matching_percentage

# Ví dụ sử dụng
new_cv_text = extract_text_from_pdf(r"C:\Users\pngoc\Downloads\Smart-Talent-Resume-Ranker-kapil-development\Smart-Talent-Resume-Ranker-kapil-development\CV\nikhilkumar9917103088 - Nikhil kumar.pdf")  # Trích xuất văn bản từ file PDF CV mới
matching_score = predict_matching_score(new_cv_text)  # Dự đoán điểm phù hợp của CV mới
print(f"Matching Score: {matching_score:.2f}%")  # In ra điểm phù hợp của CV mới

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Matching Score: 84.32%


Pipeline 3: Đánh giá mô hình dựa trên Precision và Recall

In [52]:
# Import các thư viện cần thiết
from sklearn.metrics import precision_score, recall_score  # Tính toán các chỉ số precision và recall

y_pred = model.predict(X_test).argmax(axis=1)  # Dự đoán nhãn (label) từ mô hình và lấy chỉ số của giá trị lớn nhất
precision = precision_score(y_test, y_pred, average='macro', zero_division=1)  # Tính toán chỉ số precision cho từng lớp (macro average)
recall = recall_score(y_test, y_pred, average='macro', zero_division=1)  # Tính toán chỉ số recall cho từng lớp (macro average)

print(f"Precision: {precision}")  # In ra chỉ số precision
print(f"Recall: {recall}")  # In ra chỉ số recall


279/279 ━━━━━━━━━━━━━━━━━━━━ 24s 84ms/step
Precision: 0.5936313992584782
Recall: 0.032195190947666194
